In [1]:
import pandas as pd
import numpy as np
import re
from string import punctuation
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
import pickle

In [2]:
import keras

#### Mempersiapkan Dataset

In [3]:
df = pd.read_csv("C:/Users/iqbal/Downloads/CHALLANGE PLATINUM/train_preprocess.tsv.txt",encoding="latin1",sep='\t',header=None,names=["text","label"])

In [4]:
df.isna().sum()#aman jaya

text     0
label    0
dtype: int64

#### Normalisasi Text

In [5]:
def lowercasing(paragraph):
    return paragraph.lower()

def menghilangkan_tandabaca(paragraph):
    new_paragraph = re.sub(fr'[{punctuation}]', r'', paragraph)
    return new_paragraph

In [6]:
def text_normalization(paragraph):
    paragraph = lowercasing(paragraph)
    paragraph = menghilangkan_tandabaca(paragraph)
    paragraph = re.sub(r"[ ]+",r' ',paragraph)
    return paragraph

In [7]:
train_data,test_data=train_test_split(df)
# train_data, test_data = train_test_split(df, test_size=0.2, random_state=0)
# train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=0)

In [8]:
train_data['text'] = train_data['text'].apply(lambda x: text_normalization(x))
# val_data['text'] = val_data['text'].apply(lambda x: text_normalization(x))
test_data['text'] = test_data['text'].apply(lambda x: text_normalization(x))

In [9]:
print(train_data.shape)
# print(val_data.shape)
print(test_data.shape)

(8250, 2)
(2750, 2)


#### Feature Extraction

In [10]:
# max_features = 100000
tokenizer = Tokenizer(oov_token='<UNK>')

In [11]:
tokenizer.fit_on_texts(train_data['text'])

In [12]:
train_data_tf = tokenizer.texts_to_sequences(train_data['text'])
# val_data_tf = tokenizer.texts_to_sequences(val_data['text'])
test_data_tf = tokenizer.texts_to_sequences(test_data['text'])

In [13]:
# max_len = int(np.quantile([len(x) for x in train_data_tf], 0.9))
train_padded = pad_sequences(sequences=train_data_tf, padding='post')
max_len = train_padded.shape[1]
# val_padded = pad_sequences(sequences=val_data_tf,padding='post', maxlen=max_len)
test_padded = pad_sequences(sequences=test_data_tf,padding='post', maxlen=train_padded.shape[1])

In [14]:
onehot = OneHotEncoder()

In [29]:
train_labels = onehot.fit_transform(train_data[['label']]).toarray()
# valid_labels = onehot.fit_transform(val_data[['label']]).toarray()
test_labels = onehot.fit_transform(test_data[['label']]).toarray()

In [30]:
train_labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

#### Prepare Train & Test Dataset

#### Training Model

In [31]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [32]:
max_features = len(tokenizer.index_word)
batch_size=16
output_dim=64
labels_tmp=32
# input_len = max_len
input_len =train_padded.shape[1]

In [33]:
len(tokenizer.index_word)

15506

In [34]:
# Mendapatkan jumlah unik token
num_unique_tokens = len(tokenizer.word_index)
print("Jumlah unik token dalam kamus:", num_unique_tokens)

Jumlah unik token dalam kamus: 15506


In [35]:
for token, index in tokenizer.word_index.items():
    if index >= num_unique_tokens:
        print(f"Token '{token}' memiliki indeks {index}, yang lebih besar dari jumlah unik token.")

Token 'massal' memiliki indeks 15506, yang lebih besar dari jumlah unik token.


In [36]:
# https://stackoverflow.com/questions/71207850/tensorflow-keras-not-utilizing-gpu

In [37]:
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=len(tokenizer.index_word),
                                 output_dim=output_dim, 
                                 input_length=input_len))
model.add(keras.layers.LSTM(128, dropout=0.2))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [38]:
from tensorflow.keras.optimizers import SGD
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
max_index_train=int(np.floor(train_padded.shape[0]/batch_size)*batch_size) 

In [40]:
model.fit(x=train_padded[:max_index_train], 
          y=train_labels[:max_index_train],
          batch_size=batch_size, 
          epochs=3, 
          shuffle=True,validation_split=0.15)
# maximum_index_training = int(np.floor(train_padded.shape[0]/batch_size)*batch_size)
# model.fit(x=train_padded[:maximum_index_training], 
#           y=train_labels[:maximum_index_training],
#           batch_size=batch_size, 
#           epochs=3, 
#           shuffle=False
#          )

Epoch 1/3
438/438 [==============================] - 92s 197ms/step - loss: 0.9193 - accuracy: 0.5842 - val_loss: 0.9017 - val_accuracy: 0.5866
Epoch 2/3
438/438 [==============================] - 102s 232ms/step - loss: 0.9124 - accuracy: 0.5854 - val_loss: 0.9007 - val_accuracy: 0.5866
Epoch 3/3
438/438 [==============================] - 79s 180ms/step - loss: 0.9120 - accuracy: 0.5854 - val_loss: 0.9018 - val_accuracy: 0.5866


In [43]:
from sklearn.metrics import classification_report

In [44]:
test_padded.shape

(2750, 91)

In [45]:
prediction=model.predict(test_padded,batch_size=64)
prediction

43/43 [==============================] - 6s 115ms/step


array([[0.29693985, 0.09283732, 0.61022276],
       [0.29693988, 0.09283732, 0.61022276],
       [0.29693985, 0.09283732, 0.61022276],
       ...,
       [0.2969402 , 0.09283781, 0.61022204],
       [0.29693988, 0.09283732, 0.61022276],
       [0.29693985, 0.09283732, 0.61022276]], dtype=float32)

In [46]:
len(onehot.inverse_transform(prediction))

2750

In [47]:
test_data[["label"]]

,label
8279,negative
5758,negative
5863,neutral
9014,neutral
2255,positive
...,...
8096,positive
2296,positive
6243,negative
3231,positive


In [48]:
print(classification_report(y_true=test_data[["label"]],y_pred=onehot.inverse_transform(prediction)))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       851
     neutral       0.00      0.00      0.00       312
    positive       0.58      1.00      0.73      1587

    accuracy                           0.58      2750
   macro avg       0.19      0.33      0.24      2750
weighted avg       0.33      0.58      0.42      2750



C:\Users\iqbal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\iqbal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\iqbal\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
#semua berjalan dengan lanca maka tinggal copas untuk kfold
from sklearn.model_selection import KFold
kfold=KFold(random_state=0,shuffle=True)
for train,test in kfold.split(df):
    train_data=df.loc[train]
    test_data=df.loc[test]
    onehot=OneHotEncoder()
    label=onehot.fit_transform(train_data[["label"]])
    
    #data udah ada
    train_data['text'] = train_data['text'].apply(lambda x: text_normalization(x))
    test_data['text'] = test_data['text'].apply(lambda x: text_normalization(x))
    
    tokenizer=Tokenizer(oov_token="UNK")
    tokenizer.fit_on_texts(train_data["text"])
    train_data_tf=tokenizer.texts_to_sequences(train_data["text"])
    test_data_tf=tokenizer.texts_to_sequences(test_data["text"])
    
    train_data_pad=pad_sequences(train_data_tf,padding="post")
    test_data_pad=pad_sequences(test_data_tf,padding="post",maxlen=train_data_pad.shape[1])
    
    model=Sequential()
    model.add(layers.Embedding(len(tokenizer.index_word),64,input_length=train_data_pad.shape[1]))
    model.add(layers.Conv1D(128,5,activation="relu"))
    model.add(layers.GlobalMaxPool1D())
    model.add(layers.Dense(10,activation="relu"))
    model.add(layers.Dense(3,activation="softmax"))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'],)
    
    batch_size=64
    max_index_train=int(np.floor(train_data_pad.shape[0]/batch_size)*batch_size)
    model.fit(x=train_data_pad[:max_index_train],y=label.toarray()[:max_index_train],batch_size=64,epochs=3,shuffle=True)
    prediction=model.predict(test_data_pad,batch_size=64)
    prediction=onehot.inverse_transform(prediction)
    print(classification_report(y_true=test_data[["label"]],y_pred=prediction))

Epoch 1/3
137/137 [==============================] - 20s 134ms/step - loss: 0.7348 - accuracy: 0.6902
Epoch 2/3
137/137 [==============================] - 20s 144ms/step - loss: 0.3149 - accuracy: 0.8880
Epoch 3/3
35/35 [==============================] - 1s 38ms/step
              precision    recall  f1-score   support

    negative       0.82      0.84      0.83       688
     neutral       0.82      0.72      0.77       240
    positive       0.91      0.92      0.91      1272

    accuracy                           0.87      2200
   macro avg       0.85      0.83      0.84      2200
weighted avg       0.87      0.87      0.87      2200

Epoch 1/3
137/137 [==============================] - 23s 148ms/step - loss: 0.7645 - accuracy: 0.6529
Epoch 2/3
137/137 [==============================] - 21s 155ms/step - loss: 0.3210 - accuracy: 0.8815
Epoch 3/3
35/35 [==============================] - 2s 41ms/step
              precision    recall  f1-score   support

    negative       0.78     

In [52]:
class_report=classification_report(y_true=test_data[["label"]],y_pred=prediction,output_dict=True)

In [54]:
class_report

{'negative': {'precision': 0.8403614457831325,
  'recall': 0.8454545454545455,
  'f1-score': 0.8429003021148036,
  'support': 660},
 'neutral': {'precision': 0.8082901554404145,
  'recall': 0.7722772277227723,
  'f1-score': 0.789873417721519,
  'support': 202},
 'positive': {'precision': 0.9344750558451228,
  'recall': 0.9379671150971599,
  'f1-score': 0.9362178291682207,
  'support': 1338},
 'accuracy': 0.895,
 'macro avg': {'precision': 0.8610422190228899,
  'recall': 0.8518996294248259,
  'f1-score': 0.8563305163348477,
  'support': 2200},
 'weighted avg': {'precision': 0.8946549046984571,
  'recall': 0.895,
  'f1-score': 0.8947854932739076,
  'support': 2200}}

In [55]:
formatted_report = {
    'precision': {},
    'recall': {},
    'f1-score': {},
    'support': {}
}

for class_label, metrics in class_report.items():
    if class_label in ['accuracy', 'macro avg', 'weighted avg']:
        continue
    formatted_report['precision'][class_label] = metrics['precision']
    formatted_report['recall'][class_label] = metrics['recall']
    formatted_report['f1-score'][class_label] = metrics['f1-score']
    formatted_report['support'][class_label] = metrics['support']

# Masukkan metrik keseluruhan
overall_metrics = class_report['macro avg']
formatted_report['precision']['macro avg'] = overall_metrics['precision']
formatted_report['recall']['macro avg'] = overall_metrics['recall']
formatted_report['f1-score']['macro avg'] = overall_metrics['f1-score']
formatted_report['support']['macro avg'] = overall_metrics['support']

# Masukkan metrik weighted average
weighted_metrics = class_report['weighted avg']
formatted_report['precision']['weighted avg'] = weighted_metrics['precision']
formatted_report['recall']['weighted avg'] = weighted_metrics['recall']
formatted_report['f1-score']['weighted avg'] = weighted_metrics['f1-score']
formatted_report['support']['weighted avg'] = weighted_metrics['support']

# Tambahkan accuracy
formatted_report['accuracy'] = class_report['accuracy']

# Ubah ke dalam format JSON
import json
formatted_report_json = json.dumps(formatted_report, indent=4)

# Simpan ke dalam file JSON
with open('formatted_classification_report.json', 'w') as json_file:
    json_file.write(formatted_report_json)

In [63]:
import pickle
import os
output_directory = r'C:\Users\iqbal\OneDrive\Documents\BOOTCAMP\CHALLANGE PLATINUM\LSTM'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Simpan objek 'onehot' ke dalam file
onehot_file_path = os.path.join(output_directory, 'onehot.pkl')
with open(onehot_file_path, 'wb') as onehot_file:
    pickle.dump(obj=onehot, file=onehot_file)

# Simpan objek 'tokenizer' ke dalam file
tokenizer_file_path = os.path.join(output_directory, 'tokenizer.pkl')
with open(tokenizer_file_path, 'wb') as tokenizer_file:
    pickle.dump(obj=tokenizer, file=tokenizer_file)


In [64]:
model.save('C:/Users/iqbal/OneDrive/Documents/BOOTCAMP/CHALLANGE PLATINUM/LSTM/h5/model_lstm.h5')

C:\Users\iqbal\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [65]:
maxlen=train_data_pad.shape[1]#harus diingat untuk padding inputs

In [66]:
report_lstm=classification_report(y_true=test_data[["label"]],y_pred=prediction)

In [67]:
report_lstm

'              precision    recall  f1-score   support\n\n    negative       0.84      0.85      0.84       660\n     neutral       0.81      0.77      0.79       202\n    positive       0.93      0.94      0.94      1338\n\n    accuracy                           0.90      2200\n   macro avg       0.86      0.85      0.86      2200\nweighted avg       0.89      0.90      0.89      2200\n'

In [68]:
output_directory = r'C:\Users\iqbal\OneDrive\Documents\BOOTCAMP\CHALLANGE PLATINUM\LSTM\json'

# Simpan laporan hasil LSTM ke dalam file JSON
output_file_path = os.path.join(output_directory, 'report_lstm.json')
json.dump(report_lstm, open(output_file_path, "w"), indent=4)